In [5]:
import os
import json
import pandas as pd
import requests
from dotenv import load_dotenv

from google.cloud import bigquery

load_dotenv()

True

In [6]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [7]:
def extract_videos() -> pd.DataFrame:
    project_id = os.getenv('PROJECT_ID')
    client = bigquery.Client(project=project_id)
    table_id = f'{project_id}.marts.videos'

    sql = f"""
    SELECT channel_name, title
    FROM `{table_id}`
    ORDER BY channel_name, title
    """

    df_videos = client.query(sql).to_dataframe()
    return df_videos

In [8]:
df_videos = extract_videos()

In [25]:
scope = ["user-library-read", "user-library-modify"]

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

### Priority: album -> playlist -> artist -> track

In [15]:
df = df_videos[0:5]

In [29]:
def find_spotify_ids(row):
    #row['channel_name']
    #row['title']
    results = sp.search(q=row['title'], limit=1, type='track')
    track_id = results['tracks']['items'][0]['id']
    
    sp.current_user_saved_tracks_add(tracks=[track_id])

In [30]:
df.apply(find_spotify_ids, axis = 1)

0    None
1    None
2    None
3    None
4    None
dtype: object

In [ ]:
response['tracks']['items'][0]['id'] # ['album'] ['artists'] ['available_markets'] ['duration_ms']

### Save the tracks